In [1]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import joblib  # For loading the trained model

# Load your trained model
model = joblib.load("/Users/alesarabandi/Loan-Appr-Pred/trained_model.pkl")

# Function to encode home ownership (existing logic)
def encode_home_ownership(home_ownership):
    if home_ownership == "RENT":
        return [1, 0, 0]  # RENT
    elif home_ownership == "OWN":
        return [0, 1, 0]  # OWN
    elif home_ownership == "MORTGAGE":
        return [0, 0, 1]  # MORTGAGE
    return [0, 0, 0]  # Default to all zeros if no valid input

# Function to encode car ownership (existing logic)
def encode_car_ownership(car_ownership):
    return 1 if car_ownership.lower() == "yes" else 0

# Function to encode age groups (new logic)
def encode_age_group(age):
    if age < 30:
        return [1, 0, 0, 0, 0]  # Below 30
    elif 30 <= age < 40:
        return [0, 1, 0, 0, 0]  # 31-40
    elif 40 <= age < 50:
        return [0, 0, 1, 0, 0]  # 41-50
    elif 50 <= age < 60:
        return [0, 0, 0, 1, 0]  # 51-60
    else:
        return [0, 0, 0, 0, 1]  # Above 60

# Function for preparing the model's input features
def prepare_input(age, income, home_ownership, car_ownership):
    age_encoded = encode_age_group(age)
    home_ownership_encoded = encode_home_ownership(home_ownership)
    car_ownership_encoded = encode_car_ownership(car_ownership)

    # Create the input data for prediction
    input_data = pd.DataFrame([{
        "person_age_group_Below_30": age_encoded[0],
        "person_age_group_31_40": age_encoded[1],
        "person_age_group_41_50": age_encoded[2],
        "person_age_group_51_60": age_encoded[3],
        "person_age_group_Above_60": age_encoded[4],
        "person_income_updated": income,
        "person_home_ownership_RENT": home_ownership_encoded[0],
        "person_home_ownership_OWN": home_ownership_encoded[1],
        "person_home_ownership_MORTGAGE": home_ownership_encoded[2],
        "person_car_ownership": car_ownership_encoded
    }])

    return input_data

# Define a function for making predictions
def predict_loan_status():
    try:
        # Get input values from GUI
        age = int(entry_age.get())
        income = float(entry_income.get())
        home_ownership = var_home_ownership.get()
        car_ownership = var_car_ownership.get()

        # Prepare the input data for the model
        input_data = prepare_input(age, income, home_ownership, car_ownership)

        # Make prediction
        prediction = model.predict(input_data)
        probability = model.predict_proba(input_data)[:, 1][0]  # Probability of loan approval

        # Display the result
        result = "Loan Approved" if prediction[0] == 1 else "Loan Denied"
        messagebox.showinfo("Prediction Result", f"Loan Status: {result}\nApproval Probability: {probability:.2f}")
    except Exception as e:
        messagebox.showerror("Error", f"Invalid input: {e}")

# Create the GUI window
window = tk.Tk()
window.title("Loan Approval Prediction")

# Input fields
tk.Label(window, text="Age:").grid(row=0, column=0, padx=10, pady=5)
entry_age = tk.Entry(window)
entry_age.grid(row=0, column=1, padx=10, pady=5)

tk.Label(window, text="Income:").grid(row=1, column=0, padx=10, pady=5)
entry_income = tk.Entry(window)
entry_income.grid(row=1, column=1, padx=10, pady=5)

# Home Ownership input using OptionMenu
tk.Label(window, text="Home Ownership:").grid(row=2, column=0, padx=10, pady=5)
var_home_ownership = tk.StringVar(value="RENT")
home_options = ["RENT", "OWN", "MORTGAGE"]
home_menu = tk.OptionMenu(window, var_home_ownership, *home_options)
home_menu.grid(row=2, column=1, padx=10, pady=5)

# Car Ownership input using OptionMenu
tk.Label(window, text="Car Ownership (Yes/No):").grid(row=3, column=0, padx=10, pady=5)
var_car_ownership = tk.StringVar(value="No")
car_options = ["Yes", "No"]
car_menu = tk.OptionMenu(window, var_car_ownership, *car_options)
car_menu.grid(row=3, column=1, padx=10, pady=5)

# Predict button
tk.Button(window, text="Predict", command=predict_loan_status).grid(row=4, column=0, columnspan=2, pady=10)

# Run the GUI loop
window.mainloop()


: 